In [58]:
from dateutil import rrule
from datetime import datetime
from datetime import timedelta
import string
import math
import random
# ## rrule
characters = string.ascii_letters + string.digits
def get_random_string(length: int) -> str:
    return ''.join(random.choice(characters) for i in range(length))

def get_random_date_from(start_date):
    return start_date + timedelta(days=random.randint(0,1300))
    

# function that returns random timestamps


def get_random_date():
    minutes_in_year = 365 * 24 * 60
    return datetime.now() + timedelta(minutes=random.randint(200, minutes_in_year*10))
def get_random_unique_days_of_week_list():
    days_of_weeks = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    days = []
    for i in days_of_weeks:
        if random.randint(0,200)%2 == 0:
            days.append(i)
    return days
def get_random_end(start_date):
    if random.randint(0,200)%2 == 0:
        return "Never"
    else:
        if random.randint(1, 100)%2 == 0:
            return f"{random.randint(0,100)} Occurrences"
        else :
            return get_random_date_from(start_date)
def get_random_recurrence(original_date):
    repeat_every = random.randint(1, 100)
    repeat_by = random.choice(['Day', 'Week', 'Month', 'Year'])
    repeat_on = get_random_unique_days_of_week_list()
    ends_on = get_random_end(original_date)
    return {
        "repeatsEvery": repeat_every,
        "repeatsBy": repeat_by,
        "repeatsOn": repeat_on,
        "endsOn": ends_on,
    }
def get_rrule_string(original_date:datetime,interval_type,ceil_count=100) -> str:
    date_map = {
        "Monday":"MO",
        "Tuesday":"TU",
        "Wednesday":"WE",
        "Thursday":"TH",
        "Friday":"FR",
        "Saturday":"SA",
        "Sunday":"SU",
    }
    if interval_type == "Daily":
       return f"RRULE:FREQ=DAILY;COUNT={ceil_count}"
    elif interval_type == "Weekly On Day":
        current_week = original_date.isocalendar()[1]
        day = original_date.strftime("%A")
        rrule_day = date_map[day]
        return f"RRULE:FREQ=WEEKLY;BYDAY={rrule_day};COUNT={ceil_count}"
    elif interval_type == "Monthly On Current Day Of Current Week":
        day = date_map[original_date.strftime("%A")]
        week_number=get_week_of_month(original_date)
        return f"RRULE:FREQ=MONTHLY;BYDAY={week_number}{day};COUNT={ceil_count}"
    elif interval_type == "Yearly On Current Date Of Current Month":
        day = int(original_date.strftime("%d"))
        month = int(original_date.strftime("%m"))
        return f"RRULE:FREQ=YEARLY;BYMONTH={month};BYMONTHDAY={day};COUNT={ceil_count}"
    elif interval_type == "Every Weekday":
        return f"RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT={ceil_count}"
    elif interval_type == "Custom":
        data = get_random_recurrence(original_date)
        ends_on = data["endsOn"]
        repeats_by = data["repeatsBy"]
        repeats_on = data["repeatsOn"]
        repeasts_every = data["repeatsEvery"]
    else:
        pass
        
def get_week_of_month(date):
   first_day = date.replace(day=1)

   day_of_month = date.day

   if(first_day.weekday() == 6):
       adjusted_dom = (1 + first_day.weekday()) / 7
   else:
       adjusted_dom = day_of_month + first_day.weekday()

   return int(math.ceil(adjusted_dom/7.0))


In [59]:
from faker import Faker
fk = Faker()
def generate_meeting():
    rcDetial = random.choice(["Daily", "Weekly On Day", "Monthly On Current Day Of Current Week",
                              "Yearly On Current Date Of Current Month", "Every Weekday"])
    date = get_random_date()
    rrule_string = get_rrule_string(date, rcDetial, ceil_count=random.randint(1, 100))
    return {
        "hostingSubDivisionId": get_random_string(10),
        "isParent": True,
        "rrule": rrule_string,
        "recurrenceDetail": rcDetial,
        "hasEndDate": random.choice([True, False]),
        "endDate": get_random_date_from(date).isoformat(),
        "meetingTime": date.strftime("%H:%M %p"),
        "meetingType": fk.sentence(),
    }


meeting_list = [generate_meeting() for _ in range(100)]
meeting_list[0:4]


[{'hostingSubDivisionId': 'gBtnCqWAlt',
  'isParent': True,
  'rrule': 'RRULE:FREQ=WEEKLY;BYDAY=WE;COUNT=66',
  'recurrenceDetail': 'Weekly On Day',
  'hasEndDate': False,
  'endDate': '2028-12-07T07:37:45.919101',
  'meetingTime': '07:37 AM',
  'meetingType': 'Worker for couple another true east.'},
 {'hostingSubDivisionId': '4U65mRUMAq',
  'isParent': True,
  'rrule': 'RRULE:FREQ=DAILY;COUNT=75',
  'recurrenceDetail': 'Daily',
  'hasEndDate': True,
  'endDate': '2027-03-21T03:01:45.919163',
  'meetingTime': '03:01 AM',
  'meetingType': 'Half budget avoid itself.'},
 {'hostingSubDivisionId': 'SoYd3P4zCC',
  'isParent': True,
  'rrule': 'RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT=62',
  'recurrenceDetail': 'Every Weekday',
  'hasEndDate': False,
  'endDate': '2029-11-12T21:38:45.919181',
  'meetingTime': '21:38 PM',
  'meetingType': 'Computer focus concern century material mouth offer.'},
 {'hostingSubDivisionId': '5zllmgztSX',
  'isParent': True,
  'rrule': 'RRULE:FREQ=WEEKLY;BYDAY=

In [60]:
def make_list_from_rrule(data,start_date=None,end_date=None):
    rrule_str = data["rrule"]
    rrule_dt_list = []
    final_list = []
    if start_date is None:
        rrule_dt_list = list(rrule.rrulestr(rrule_str))                
    else:
        rrule_dt_list = list(rrule.rrulestr(rrule_str,dtstart=start_date,until=end_date))
    for i in rrule_dt_list:
        final_list.append(
            {
                **data,
                "meetingDate": i.isoformat(),
            }
        )
    return final_list

        

In [61]:
from concurrent.futures import ThreadPoolExecutor,as_completed
import pprint
def get_final_unsorted_list():
    unsorted_list = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_data = {executor.submit(make_list_from_rrule,data):data for data in meeting_list}
        for future in as_completed(future_to_data):
            data = future_to_data[future]
            final_list = future.result()
            unsorted_list.extend(final_list)
    return unsorted_list
def sort_meeting_by_date(unsorted_arr):
    sorted_list = sorted(unsorted_arr, key=lambda k: k['meetingDate'])
    return sorted_list
un_data = get_final_unsorted_list()
sorted_data = sort_meeting_by_date(un_data)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(len(sorted_data))
import json
#save to file
with open("meeting_data.json","w") as f:
    json.dump(sorted_data,f)


4832


In [62]:

import re
import pytz
current_date = f'{datetime.now().isoformat().replace("-", "").replace(":", "")}Z'
string_d = 'RRULE:FREQ=WEEKLY;BYDAY=MO,TU,WE,TH,FR;COUNT=79'
final_string = re.sub(r'COUNT=\d*', f'UNTIL={current_date}', string_d)
list(rrule.rrulestr(final_string))
def make_list_from_rrule(data):
    dt = data[0]
    rng = data[1]
    original_date= data[2]
    start_date = original_date.astimezone(pytz.UTC)
    end_date = original_date 
    rrule_str = dt["rrule"]
    if rng == "D":
        end_date = original_date + timedelta(days=1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    elif rng == "W":
        start_date = original_date - timedelta(days=original_date.weekday())
        end_date = start_date + timedelta(weeks=1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    elif rng == "M":
        start_date = original_date.replace(day=1)
        end_date = start_date.replace(month=original_date.month+1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    elif rng == "Y":
        start_date = original_date.replace(day=1,month=1)
        end_date = start_date.replace(year=original_date.year+1)
        date_iso_string = f'{end_date.isoformat().replace("-", "").replace(":", "")}Z'
        rrule_str = re.sub(r"COUNT=\d*", f'UNTIL={date_iso_string}', rrule_str)
    final_list = []
    date_iso_string = f'{start_date.isoformat().replace("-", "").replace(":", "")}Z'
    rrule_str = f"DTSTART:{date_iso_string}\n"+rrule_str
    # print(rrule_str)
    rrule_dt_list = list(rrule.rrulestr(rrule_str,dtstart=start_date))
    for i in rrule_dt_list:
        final_list.append(
            {
                **dt,
                "meetingDate": i.isoformat()
            }
        )
    return final_list
    

def get_meeting_data_by_range(date,rg=None):
    # rg = D for Date, M for month, Y for year
    # date = date of choice
    unsorted_list = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_data = {executor.submit(make_list_from_rrule,[data,rg,date]):data for data in meeting_list}
        for future in as_completed(future_to_data):
            data = future_to_data[future]
            final_list = future.result()
            unsorted_list.extend(final_list)
    return sort_meeting_by_date(unsorted_list)


            
    

In [63]:
june_data = get_meeting_data_by_range(datetime.now(),'M')
# regex pattern for finding number between decimal 50 to decimal 2000

 

# Meeting On Month Of June


In [64]:

for i in june_data:
    print(f"{i['meetingDate']} - {i['meetingTime']} - {i['meetingType']}")

2022-06-01T18:19:46+00:00 - 16:56 PM - Check modern south happen.
2022-06-01T18:19:46+00:00 - 14:42 PM - Young could line goal somebody.
2022-06-01T18:19:46+00:00 - 18:19 PM - Avoid have on single exactly.
2022-06-01T18:19:46+00:00 - 07:37 AM - Worker for couple another true east.
2022-06-01T18:19:46+00:00 - 13:45 PM - Nation important left responsibility enough.
2022-06-01T18:19:46+00:00 - 16:26 PM - Once Mrs without nothing truth painting.
2022-06-01T18:19:46+00:00 - 06:40 AM - Campaign media may surface.
2022-06-01T18:19:46+00:00 - 03:01 AM - Half budget avoid itself.
2022-06-01T18:19:46+00:00 - 05:38 AM - Lose Democrat however still model TV.
2022-06-01T18:19:46+00:00 - 05:41 AM - Follow strong know.
2022-06-01T18:19:46+00:00 - 17:58 PM - Method most specific onto discover will.
2022-06-01T18:19:46+00:00 - 10:18 AM - Group star push project investment human west.
2022-06-01T18:19:46+00:00 - 12:37 PM - Church cause provide heart.
2022-06-01T18:19:46+00:00 - 06:07 AM - All early brin

# Meeting This Week On June

In [66]:
week_data = get_meeting_data_by_range(datetime.now(),'W')
for i in week_data:
    print(f"{i['meetingDate']} - {i['meetingTime']} - {i['meetingType']}")


2022-06-13T18:20:04+00:00 - 07:07 AM - Themselves well our after never her treat note.
2022-06-13T18:20:04+00:00 - 21:38 PM - Computer focus concern century material mouth offer.
2022-06-13T18:20:04+00:00 - 16:26 PM - Once Mrs without nothing truth painting.
2022-06-13T18:20:04+00:00 - 16:54 PM - Drug cover partner everyone he rate hundred.
2022-06-13T18:20:04+00:00 - 02:33 AM - Toward and rule admit still allow late.
2022-06-13T18:20:04+00:00 - 13:46 PM - Clearly produce stage majority something it.
2022-06-13T18:20:04+00:00 - 23:19 PM - Increase discover option senior in.
2022-06-13T18:20:04+00:00 - 05:41 AM - Follow strong know.
2022-06-13T18:20:04+00:00 - 16:56 PM - Check modern south happen.
2022-06-13T18:20:04+00:00 - 06:40 AM - Campaign media may surface.
2022-06-13T18:20:04+00:00 - 12:37 PM - Church cause provide heart.
2022-06-13T18:20:04+00:00 - 03:35 AM - Again understand walk above specific.
2022-06-13T18:20:04+00:00 - 17:58 PM - Method most specific onto discover will.
202